In [2]:
import openpyxl
import pandas as pd

pd.set_option('display.max_colwidth', None)  # None para largura ilimitada
pd.set_option('display.width', None)

def table(excelname, sheetname):
    workbook = openpyxl.load_workbook(excelname + '.xlsx', data_only=True)
    sheet = workbook[sheetname]

    dados_filtrados = []
    cabecalhos = []
    num_colunas = 11

    # Ler APENAS os 11 primeiros cabeçalhos da primeira linha
    for i in range(1, num_colunas + 1):
        cabecalho = sheet.cell(row=1, column=i).value
        cabecalhos.append(cabecalho)

    print(f"Número de cabeçalhos lidos: {len(cabecalhos)}")
    print(f"Cabeçalhos: {cabecalhos}")

    # Ler os dados a partir da segunda linha até a linha 45391 e filtrar
    for linha_num in range(2, 45392):
        linha_dados = []
        for coluna in range(1, num_colunas + 1):  # Colunas de 1 (A) a 11 (K)
            celula = sheet.cell(row=linha_num, column=coluna)
            linha_dados.append(celula.value)

        if len(linha_dados) == num_colunas:
            dados_filtrados.append(linha_dados)

    return cabecalhos, dados_filtrados

excelname = 'ROI_v2'
sheetname = 'BASE'  
cabecalhos, dados_filtrados = table(excelname, sheetname)

if dados_filtrados:
    try:
        df = pd.DataFrame(dados_filtrados, columns=cabecalhos)
        print("\nDataFrame criado com as linhas filtradas:")
        

        # Formatar colunas de venda para moeda
        colunas_venda = ['VENDAS LT', 'VENDAS VALOR R$', 'R$ INV.']
        for col in colunas_venda:
            if col in df.columns:
                # Formata para moeda com 2 casas decimais
                # Usando um formato de string mais direto
                df[col] = df[col].apply(lambda x: "{:,.2f}".format(x) if pd.notnull(x) else '')

        print("\nDataFrame com formatação de moeda nas colunas de venda:")
        print(df.head())

        # Tenta converter colunas para datetime, se aplicável
        for col in df.columns:
            try:
                df[col] = pd.to_datetime(df[col])
            except (ValueError, TypeError):
                pass 

        
        data_columns = ['PERÍODO'] 
        for col in data_columns:
            if col in df.columns:
                df[col] = df[col].dt.strftime('%d/%m/%Y') 
        

    except ValueError as e:
        print(f"\nErro ao criar o DataFrame com os dados filtrados: {e}")
        print(f"Número de cabeçalhos: {len(cabecalhos)}")
        if dados_filtrados:
            print(f"Número de colunas na primeira linha dos dados filtrados: {len(dados_filtrados[0])}")

c:\Users\henri\anaconda3\lib\site-packages\openpyxl\reader\excel.py:228: UserWarning: Slicer List extension is not supported and will be removed
  ws_parser.bind_all()


Número de cabeçalhos lidos: 11
Cabeçalhos: ['UF/ÁREA', 'FAMÍLIA', 'PERÍODO', 'VENDAS LT', 'VENDAS VALOR R$', 'SHARE VOLUME', 'SHARE VALOR', 'DISTRIBUIÇÃO NUMÉRICA', 'R$ INV.', 'SEGMENTO', 'ANO MÓVEL']

DataFrame criado com as linhas filtradas:

DataFrame com formatação de moeda nas colunas de venda:
  UF/ÁREA FAMÍLIA    PERÍODO   VENDAS LT VENDAS VALOR R$  SHARE VOLUME  \
0      AL  AMSTEL 2018-01-01  139,432.73    1,460,088.01      0.001710   
1      AL  AMSTEL 2018-02-01   78,649.51    1,448,212.95      0.001593   
2      AL  AMSTEL 2018-03-01   82,657.57    1,614,079.91      0.000054   
3      AL  AMSTEL 2018-04-01  127,809.49    1,455,398.53      0.000090   
4      AL  AMSTEL 2018-05-01   88,741.58    1,243,697.54      0.000001   

   SHARE VALOR  DISTRIBUIÇÃO NUMÉRICA     R$ INV.    SEGMENTO ANO MÓVEL  
0     0.001556                     60   53,653.18  MAINSTREAM         0  
1     0.001512                     47  245,058.95  MAINSTREAM    MOV'19  
2     0.000055                  